**Описание проекта**  
Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.  
Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.  
В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».


**Описание данных**
- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы проданных копий)
- EU_sales — продажи в Европе (миллионы проданных копий)
- JP_sales — продажи в Японии (миллионы проданных копий)
- Other_sales — продажи в других странах (миллионы проданных копий)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.
Данные за 2016 год могут быть неполными.

**Шаг 1. Откроем файл с данными и изучим общую информацию**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import math
import seaborn as sns

In [ ]:
games = pd.read_csv('/datasets/games.csv')
games.head()

In [ ]:
games.describe()

In [ ]:
games.info()

Оценим процент NaN в столбцах.

In [ ]:
games.isnull().sum().sort_values(ascending = False)/games.shape[0]*100

Столбец Critic_Score. Оценку критиков мы не можем просчитать или восстановить, заполним для удобства NaN произвольным большим числом 9999. Так же поступим со столбцом user_score и приведем его к float. Не забыть == 'tbd', это те же пропуски.

Столбец Year_of_Release содержит пропуски и не является целочисленным, также заполним пропуски произвольным большим числом, а затем - преобразуем к нужному типу.  

In [ ]:
rating = pd.read_csv('/datasets/rating.csv')
rating.head()

In [ ]:
rating.describe()

In [ ]:
rating.info()

**Шаг 2. Подготовим данные**

Заменим названия столбцов (приведем к нижнему регистру).

In [ ]:
games.columns = games.columns.str.lower()
games.columns.tolist()

Посмотрим, какие типы данных нужно преобразовать. NaN в столбце Year_of_Release мы не можем заполнить с помощью каких-либо умозаключений, поэтому заполним их каким-нибудь большим числом, преобразуем данные столбца к нужному типу. Так же заполним для удобства большим числом NaN в столбцах critic_score и user_score.

In [ ]:
games['year_of_release'] = games['year_of_release'].fillna(9999).astype('int')
games['critic_score']=games['critic_score'].fillna(9999)
games['user_score']=games['user_score'].fillna(9999)

Обратим внимание на аббревиатуру 'tbd' в столбцах с рейтингом.

In [ ]:
games[games['user_score'] == 'tbd']['user_score'].count()

TBD — сокращение от To Be Determined. Значение не определено, значит, это те же пропуски, поставим вместо них большое число.

In [ ]:
games.loc[games['user_score'] == 'tbd','user_score'] = 9999
games['user_score'] = games['user_score'].astype('float')

В текстовых столбцах заполним NaN соответствующими категориями.

In [ ]:
games['rating'] = games['rating'].fillna('no rating')
games['genre'] = games['genre'].fillna('no genre')
games['name'] = games['name'].fillna('no name')

Перед проверкой на дубликаты переведем все значения текстовых полей в нижний регистр.

In [ ]:
games['name']=games['name'].str.lower()
games['platform']=games['platform'].str.lower()
games['genre']=games['genre'].str.lower()
#games['rating']=games['rating'].str.lower()
#rating['rating_type]=rating['rating_type'].str.lower()
rating['decryption']=rating['decryption'].str.lower()

In [ ]:
games.duplicated().sum()

In [ ]:
rating.duplicated().sum()

In [ ]:
games.info()

Посчитаем количество продаж во всех регионах и запишем в отдельный столбец.

In [ ]:
games['total_sales'] = games['na_sales'] + games['eu_sales'] + games['jp_sales'] + games['other_sales']
games.head()

Выполнили предобработку. 

**Шаг 3. Проведем исследовательский анализ данных**

- Посмотрим, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?
- Посмотрим, как менялись продажи по платформам. Выберем платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?
- Возьмем данные за соответствующий актуальный период. Актуальный период определим самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год.
- Не учитываем в работе данные за предыдущие годы.
- Какие платформы лидируют по продажам, растут или падают? Выберем несколько потенциально прибыльных платформ.
- Построим график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишем результат.
- Посмотрим, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Построим диаграмму рассеяния и посчитаем корреляцию между отзывами и продажами. Сформулируем выводы.
- Соотнесем выводы с продажами игр на других платформах.
- Посмотрим на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

Посмотрим, сколько игр выпускалось в разные годы.

In [ ]:
def plot_bar(df, column, title):
    df_not_na = df[df[column] != 9999]
    df_plot = df_not_na.groupby(column)[column].count()
    ax = df_plot.plot(kind='bar', figsize=(12, 5)).legend()
    plt.title(title)
    plt.show()

plot_bar(games, 'year_of_release', 'Распределение релиза игр по годам')


Мы видим, что первые официальные релизы игр появились в 80-х годах, их количество до 90-х годов не превышало определенных значений. Постепенный рост выпуска игр произошел в 90-х, что можно объяснить ростом возможностей операционных систем, массовым распространением ПК среди пользователей. Пик выпуска игр приходится на 2008-2009 годы, далее следует спад, который можно объяснить распределением сферы интересов потребителей игровой индустрии среди прочих гаджетов.

Посмотрим, как менялись продажи по платформам. Выберем платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?

In [ ]:
def plot_bar(df, column_gr, column_sum, title):
    df_plot = df.groupby(column_gr)[column_sum].sum().sort_values(ascending = False)
    ax = df_plot.plot(kind='bar', figsize=(12, 5)).legend()
    plt.title(title)
    plt.show()

plot_bar(games, 'platform', 'total_sales', 'Распределение продаж игр по платформам')

Лидером является платформа PS2. Далее идут Xbox360, PS3 и Wii, PS замыкает шестерку явных лидеров.

Выберем топ-6 платформ с наибольшими суммарными продажами и построим распределение по годам.

In [ ]:
games_gr=games.pivot_table(index='platform', values = 'total_sales',aggfunc = sum).query('total_sales > 700').sort_values(by = 'total_sales', ascending = False)
print(games_gr)
#head()

In [ ]:
top6 = ['ps2', 'x360', 'ps3', 'wii', 'ds', 'ps']

In [ ]:
ax = plt.gca()
legend=[]
for platform in top6:
    df_plot = games.query('platform == @platform')
    df_plot.groupby('year_of_release')['total_sales'].sum().plot(x='year_of_release', y='total_sales', xlim=(1980, 2016), ylim=(0, 300), style='.-',grid=True, kind='line', linewidth=2, ax=ax, figsize=(10,5)).legend(legend)
    legend.append(platform)
plt.show()



Игровые платформы имеют жизненный цикл порядка десяти лет. Большинство из них имеют по два пика популярности за всю историю существования, но время идет, появлятся новые платформы, которые вытесняют технически устаревшие консоли.

Возьмем данные за соответствующий актуальный период. Актуальный период определим самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год.

Актуальным обозначим период в 10 лет - с 2006 года по 2016, большинство игр из нашего топ-6 в это время активно продавались. Построим графики для этого периода.

In [ ]:
ax = plt.gca()
legend=[]
platform_list=games.query('year_of_release >= 2006')['platform'].unique().tolist()
for platform in platform_list:
    df_plot = games.query('platform == @platform')
    df_plot.groupby('year_of_release')['total_sales'].sum().plot(x='year_of_release', y='total_sales', xlim=(2006, 2016), ylim=(0, 300), style='.-',grid=True, kind='line', linewidth=2, ax=ax, figsize=(10,5)).legend(legend, loc='upper left')
    legend.append(platform)
plt.show()

На конец исследуемого периода мы можем выделить трех лидеров - PS4, Xone и 3DS, на них и следует сделать ставки в 2017 году. Также следует отметить, что продажи по всем играм на конец 2016 снижаются.

Построим график «ящик с усами» по глобальным продажам игр и в разбивке по платформам. Опишем результат.

In [ ]:
games.describe()

In [ ]:
plot = games.boxplot('total_sales', figsize=(8, 8)).set_ylim(0, 2)

Основная совокупность суммарной выручки рынка игр в верхней границе диапазона укладывается в границах 1,1 млн долларов. Медианное значение - 0,17 млн долларов.

In [ ]:
# df_plot = games.query("platform == @top6[@rows+@cols]")

platform_list=['ps2', 'x360', 'ps3', 'wii', 'ds', 'ps']
df_plot = games.query('platform in @platform_list')
ax = df_plot.boxplot(column='total_sales', by='platform', fontsize=45, figsize=(50,15), notch=True)
ax.set_ylim(0, 2)
ax.set_xlabel('')
ax.set_title('')
plt.suptitle("Boxplot продаж по топ-6", fontsize=45)
plt.show()

Медиана значений суммарных продаж игр для платформ из топ-6 принимает значения от 0,1 млн  до 0,25 млн долларов.
Верхняя граница основной совокупности значений у платформ PS3 и X360 заметно выше остальных - на уровне 1,7 млн долларов.
У остальных - от 0,6 до 1,5 миллионов долларов.

Посмотрим, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Построим диаграмму рассеяния и посчитаем корреляцию между отзывами и продажами. В качестве популярной платформы возьмем X360.

In [ ]:
games_x360 = games[(games['platform'] == 'x360') & (games['critic_score'] != 9999)].copy()
games_x360.plot(kind='scatter', x='critic_score', y='total_sales',  figsize=(10,5))

In [ ]:
corr = games_x360[['total_sales', 'critic_score']].corr()
corr.style.format("{:.2%}")

С ростом рейтинга критиков растет суммарный уровень выручки от продаж игр, связь не такая сильная, но она есть, оба фактора подтверждают качество продукта.

In [ ]:
games_x360 = games[(games['platform'] == 'x360') & (games['user_score'] != 9999)].copy()
games_x360.plot(kind='scatter', x='user_score', y='total_sales',  figsize=(10,5))

In [ ]:
corr = games_x360[['total_sales', 'user_score']].corr()
corr.style.format("{:.2%}")

Здесь картина аналогичная, с ростом продаж рейтинг отзывов пользователей растет, но связь еще слабее, чем в предыдущем случае, возможно, пользователи, зачастую, чувствуют себя свободнее критиков в плане высказывания собственного мнения.

Соотнесем выводы с продажами игр на других платформах.

In [ ]:
games_x360 = games[(games['platform'] == 'x360') & (games['critic_score'] != 9999) & (games['user_score'] != 9999)].copy()

In [ ]:
games_ps3_filtered = games_x360[['total_sales' ,'critic_score', 'user_score']]

In [ ]:
platform_list=['ps2', 'x360', 'ps3', 'wii', 'ds', 'ps']
rows = 2
cols = 3

fig, axes = plt.subplots(rows, cols, figsize=(15,10))
count=0        
for r in range(rows):
    for c in range(cols):
        platform = platform_list[count]
        df_not_na = games[(games['platform'] == platform) & (games['user_score'] != 9999) & (games['critic_score'] != 9999)].copy()
        df_final = df_not_na[['total_sales' ,'critic_score', 'user_score']]
        ax = sns.heatmap(df_final.corr()[['total_sales']].sort_values(by='total_sales', ascending=False), cmap="Blues", 
            annot=True, annot_kws={'size':10}, ax=axes[r,c])
        ax.set_title(platform, fontsize=20)
        count += 1
        

Если рассматривать корреляцию значений суммарных продаж на примере топ-6 платформ, можно заметить, что в большинстве они совпадают с результатами исследования данных по платформе X360. На фоне остальных выделяются платформы wii и ds, их продажи еще менее зависят от мнения критиков.

Посмотрим на общее распределение игр по жанрам.

In [ ]:
ax = plt.gca()
plot = games.groupby('genre').agg({'total_sales': 'sum'}).sort_values(by='total_sales', ascending=False)
plot = plot['total_sales']
plot.plot(kind='bar', figsize=(15,5), ax=ax)
ax.legend(["Общая сумма продаж"])
plt.show()

Action бесспорный лидер среди жанров.

**Шаг 4. Составим портрет пользователя каждого региона**

Самые популярные платформы (топ-5).

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(25,5))

plot = games.groupby('platform').agg({'na_sales':'sum'}).sort_values(by='na_sales', ascending=False).head(5).plot(kind='bar', ax=axes[0], fontsize=15).legend(["Общая сумма продаж, NA"])
plot = games.groupby('platform').agg({'eu_sales':'sum'}).sort_values(by='eu_sales', ascending=False).head(5).plot(kind='bar', ax=axes[1], fontsize=15).legend(["Общая сумма продаж, EU"])
plot = games.groupby('platform').agg({'jp_sales':'sum'}).sort_values(by='jp_sales', ascending=False).head(5).plot(kind='bar', ax=axes[2], fontsize=15).legend(["Общая сумма продаж, JP"])

Самые популярные жанры (топ-5).

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(25,5))

plot = games.groupby('genre').agg({'na_sales':'sum'}).sort_values(by='na_sales', ascending=False).head(5).plot(kind='bar', ax=axes[0], fontsize=15).legend(["Общая сумма продаж, NA"])
plot = games.groupby('genre').agg({'eu_sales':'sum'}).sort_values(by='eu_sales', ascending=False).head(5).plot(kind='bar', ax=axes[1], fontsize=15).legend(["Общая сумма продаж, EU"])
plot = games.groupby('genre').agg({'jp_sales':'sum'}).sort_values(by='jp_sales', ascending=False).head(5).plot(kind='bar', ax=axes[2], fontsize=15).legend(["Общая сумма продаж, JP"])

В Японии выбирают ролевые игры, лидирующие в Европе и Северной Америке жанры action и sports отодвинуты на шаг назад. Жанр shooter замыкает тройки лидеров в Европе и Северной Америке.

Влияет ли рейтинг ESRB на продажи в отдельном регионе?

In [ ]:
games_copy = games.copy()
games_copy.rename(columns = {'rating':'rating_type'}, inplace = True)
games_copy.head()


In [ ]:
#left_on = 'tariff' right_on = 'tariff_name'
games_copy = games_copy.merge(rating, on='rating_type', how='right')
games_copy.head()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(25,6))
games_copy.groupby(by='decryption').agg({'jp_sales':'sum'}).sort_values(by='jp_sales', ascending=False).head(5).plot(kind='bar', ax=axes[0], rot=30, fontsize=15)
games_copy.groupby(by='decryption').agg({'eu_sales':'sum'}).sort_values(by='eu_sales', ascending=False).head(5).plot(kind='bar', ax=axes[1], rot=30, fontsize=15)
games_copy.groupby(by='decryption').agg({'na_sales':'sum'}).sort_values(by='na_sales', ascending=False).head(5).plot(kind='bar', ax=axes[2], rot=30, fontsize=15)

Категория "для всех" лидирует во всех исследуемых регионах. В Японии аудитория подростков 13-19 представлена гораздо шире, чем в других регионах и идет на втором месте после "для всех". В Европе категория "от 17 лет" занимает второе место, но отрыв от аудитории "подростками 13-19 лет" минимален.

**Шаг 5. Проверьте гипотезы**

- Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;
- Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

H0: средний пользовательский рейтинг платформ Xbox One и PC одинаковый  
H1: средний пользовательский рейтинг платформ Xbox One и PC различается

Для проверки обеих гипотез будем использовать t-критерий Стьюдента, так как в обоих случаях выборки независимы между собой.

In [ ]:
alpha = 0.05

In [ ]:
xbox = games[(games['platform'] == 'xone') & (games['user_score'] > 0) & (games['user_score'] != 9999)]['user_score']
pc = games[(games['platform'] == 'pc') & (games['user_score'] > 0) & (games['user_score'] != 9999)]['user_score']

In [ ]:
results = st.ttest_ind(xbox, pc)

In [ ]:
print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Таким образом, пользовательский рейтинг платформ Xbox One и PC различается.

H0: средний пользовательский рейтинг жанров Action и Sports одинаковый  
H1: средний пользовательский рейтинг жанров Action и Sports различается

In [ ]:
action = games[(games['genre'] == 'action') & (games['user_score'] > 0) & (games['user_score'] != 9999)]['user_score']
sports = games[(games['genre'] == 'sports') & (games['user_score'] > 0) & (games['user_score'] != 9999)]['user_score']

In [ ]:
results = st.ttest_ind(action, sports)

In [ ]:
print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Остается верным утверждение, что средние пользовательские рейтинги жанров Action и Sports одинаковые.

В результате проведенного анализа можно сделать выводы о том, что средний срок жизни консоли составляет 10 лет, это следует учитывать при закладке бюджета разработки игр. На конец исследуемого периода мы можем выделить трех консольных лидеров - PS4, Xone и 3DS, на них и следует сделать ставки в 2017 году, однако консоль 3DS подходит к десятилетнему рубежу существования и ее продажи имеют тенденцию к снижению дольше остальных. Также важно отметить, что рынок игр в целом переживает спад, интересы аудитории сместились в сторону более актуальных гаджетов и площадок. Выходя на рынок в том или ином регионе следует учитывать популярнось жанра, платформы в целом, а также активность возрастных категорий аудитории, так как каждый регион имеет свои особенности.

In [ ]:
s1 = games[(games['genre'] == 'action') & (games['user_score'] > 0) & (games['user_score'] != 9999)]['user_score']
s2 = games[(games['genre'] == 'sports') & (games['user_score'] > 0) & (games['user_score'] != 9999)]['user_score']

In [ ]:
def checkHypothesis(s1, s2, alpha=.05):
    def makeHist(x):
        avg = np.round(x.mean(), 2)
        plt.hist(x)
        plt.title(x.name + ' ' + str(avg))
        plt.show()
    makeHist(s1)
    makeHist(s2)
    #results = st.ttest_ind(s1, s2)
    s1_std = np.std(s1)
    s2_std = np.std(s2)
    std_diff = abs(s1_std/s2_std - 1)
    eq_var = std_diff <= .05
    results = st.ttest_ind(s1, s2, equal_var=eq_var)
    print('p-значение: ', results.pvalue)
    if results.pvalue < alpha:
        print('Отвергаем нулевую гипотезу, так как различия статистически значимы')
    else:
        print('Нет оснований отвергнуть нулевую гипотезу')
    
    if eq_var:
        print('дисперсия выборок < 5 %::',eq_var)
    else:
        print('дисперсия выборок > 5 %:', std_diff)

In [ ]:
s1.name = 'action'
s2.name = 'sports'

In [ ]:
checkHypothesis(s1, s2)